In [1]:
%load_ext autotime

import os, math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from tcn import TCN
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import Input, Model
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    batch_size, timesteps, input_dim = 1, 1, input_dim
    i = Input(batch_shape=(batch_size, timesteps, input_dim))
    o = TCN(return_sequences = False)(i)
    o = Dense(1)(o)

    model = Model(inputs=[i], outputs=[o])
    model.compile(optimizer = 'adam', loss = 'mse')

    x, y = scaled_X_train_data, scaled_y_train_data
    model.fit(x, y, epochs = 20)
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 1)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

time: 2.82 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/60_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/60_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/60_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/tcn/'+file[0:7]+'result_60min.csv')

Train on 1103 samples
Epoch 1/20
1103/1103 [==============================] - 11s 10ms/sample - loss: 0.3987
Epoch 2/20
1103/1103 [==============================] - 8s 8ms/sample - loss: 0.0429
Epoch 3/20
1103/1103 [==============================] - 8s 8ms/sample - loss: 0.0283
Epoch 4/20
1103/1103 [==============================] - 9s 8ms/sample - loss: 0.0332
Epoch 5/20
1103/1103 [==============================] - 10s 9ms/sample - loss: 0.0299
Epoch 6/20
1103/1103 [==============================] - 10s 9ms/sample - loss: 0.0216
Epoch 7/20
1103/1103 [==============================] - 10s 9ms/sample - loss: 0.0173
Epoch 8/20
1103/1103 [==============================] - 9s 8ms/sample - loss: 0.0171
Epoch 9/20
1103/1103 [==============================] - 9s 8ms/sample - loss: 0.0166
Epoch 10/20
1103/1103 [==============================] - 9s 8ms/sample - loss: 0.0143
Epoch 11/20
1103/1103 [==============================] - 9s 8ms/sample - loss: 0.0156
Epoch 12/20
1103/1103 [=============

1055/1055 [==============================] - 6s 6ms/sample - loss: 0.0106
Epoch 12/20
1055/1055 [==============================] - 6s 6ms/sample - loss: 0.0091
Epoch 13/20
1055/1055 [==============================] - 6s 6ms/sample - loss: 0.0098
Epoch 14/20
1055/1055 [==============================] - 6s 6ms/sample - loss: 0.0094
Epoch 15/20
1055/1055 [==============================] - 6s 6ms/sample - loss: 0.0091
Epoch 16/20
1055/1055 [==============================] - 6s 6ms/sample - loss: 0.0093
Epoch 17/20
1055/1055 [==============================] - 6s 6ms/sample - loss: 0.0089
Epoch 18/20
1055/1055 [==============================] - 6s 6ms/sample - loss: 0.0087
Epoch 19/20
1055/1055 [==============================] - 6s 6ms/sample - loss: 0.0296
Epoch 20/20
1055/1055 [==============================] - 6s 6ms/sample - loss: 0.0094
544-ws-training.csv - 118.34016668697706
544-ws-training.csv - 28.73334791625707
Train on 1092 samples
Epoch 1/20
1092/1092 [===========================

In [4]:
rmse_list

{'584-ws-training.csv': 36.58627214060872,
 '567-ws-training.csv': 36.342665258311776,
 '596-ws-training.csv': 27.949341801716947,
 '552-ws-training.csv': 27.050176368016707,
 '544-ws-training.csv': 118.34016668697706,
 '540-ws-training.csv': 38.65145942671502}

time: 4.13 ms


In [5]:
mae_list

{'584-ws-training.csv': 27.281584626498926,
 '567-ws-training.csv': 25.704366561534126,
 '596-ws-training.csv': 21.195401124500087,
 '552-ws-training.csv': 19.91595172236501,
 '544-ws-training.csv': 28.73334791625707,
 '540-ws-training.csv': 30.241952693360574}

time: 8.46 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 47.48668028039104
Standard Deviation - RMSE : 31.991259011690556
Mean - MAE : 25.512100774085965
Standard Deviation - MAE : 3.7830047942518896
time: 5.78 ms
